In [1]:
import ee
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import agrigee_lite as agl

np.set_printoptions(suppress=True)
np.set_printoptions(precision=2)
pd.set_option('display.float_format', '{:.2f}'.format)

In [2]:
ee.Initialize(opt_url="https://earthengine-highvolume.googleapis.com", project="ee-paulagibrim")

In [4]:
df = agl.get_all_tasks()

In [13]:
set(df.description.apply(lambda x: x.split("_", 1)[0] + "_" + x.split("_", 2)[2]).tolist())

{'agl_multiple_sits_d380c64a17a6d917e4fc9a186b5b776a0bf78ac1_0',
 'agl_multiple_sits_d380c64a17a6d917e4fc9a186b5b776a0bf78ac1_1',
 'agl_multiple_sits_d380c64a17a6d917e4fc9a186b5b776a0bf78ac1_2',
 'agl_multiple_sits_d380c64a17a6d917e4fc9a186b5b776a0bf78ac1_3',
 'agl_multiple_sits_d380c64a17a6d917e4fc9a186b5b776a0bf78ac1_4',
 'agl_multiple_sits_d380c64a17a6d917e4fc9a186b5b776a0bf78ac1_5',
 'agl_multiple_sits_d380c64a17a6d917e4fc9a186b5b776a0bf78ac1_6',
 'download_samples'}

In [16]:
gdf = gpd.read_parquet("tests/data/gdf.parquet")

In [17]:
gdf

,geometry,start_date,end_date
0,"POLYGON ((-120.45276 35.72507, -120.45344 35.7...",2023-01-01,2024-01-01
1,"POLYGON ((-120.58784 35.61832, -120.5876 35.61...",2023-01-01,2024-01-01
2,"POLYGON ((-119.66351 36.14519, -119.66347 36.1...",2023-01-01,2024-01-01
3,"POLYGON ((-120.6177 36.87609, -120.61779 36.86...",2023-01-01,2024-01-01
4,"POLYGON ((-120.67999 37.74592, -120.68003 37.7...",2023-01-01,2024-01-01
...,...,...,...
69801,"POLYGON ((-121.04369 38.43856, -121.04439 38.4...",2023-01-01,2024-01-01
69802,"POLYGON ((-119.84432 36.72782, -119.84435 36.7...",2023-01-01,2024-01-01
69803,"POLYGON ((-119.25738 36.44657, -119.25743 36.4...",2023-01-01,2024-01-01
69804,"POLYGON ((-120.89781 37.64557, -120.89787 37.6...",2023-01-01,2024-01-01


In [ ]:
gdf = agl.quadtree_clustering(gdf)

In [ ]:


username = getpass.getuser()
print(username)

mateus


In [20]:
from agrigee_lite.misc import create_gdf_hash

In [24]:
hashname = create_gdf_hash(gdf)

In [ ]:
satellite = agl.sat.Sentinel2(use_sr=True, selected_bands=["red", "green", "blue"])

In [32]:
for cluster_id in sorted(gdf.cluster_id.unique()):
    cluster_id = int(cluster_id)
    agl.get.multiple_sits_task_gdrive(gdf[gdf.cluster_id == cluster_id], satellite, f"{hashname}_{cluster_id}", f"agl_mateus_multiple_sits_{hashname}_{cluster_id}")

    if cluster_id > 5:
        break

In [6]:
row = gdf.iloc[0]

In [7]:
sits = agl.get.sits(row.geometry, row.start_date, row.end_date, satellite)

In [8]:
sits

,indexnum,s2sr_doy_0,s2sr_doy_1,s2sr_doy_2,s2sr_doy_3,s2sr_doy_4,s2sr_doy_5,s2sr_doy_6,s2sr_doy_7,s2sr_doy_8,...,s2sr_blue_45,s2sr_blue_46,s2sr_blue_47,s2sr_blue_48,s2sr_blue_49,s2sr_blue_50,s2sr_blue_51,s2sr_blue_52,s2sr_blue_53,s2sr_blue_54
0,0,1,6,21,26,31,36,41,46,61,...,706,657,733,650,1019,725,1385,690,745,682


In [ ]:
sits_2

In [ ]:
df_wide = pd.DataFrame(sits_2.values.flatten().reshape(1, -1))
df_wide.columns = [f"{col}_{i}" for i in range(len(sits_2)) for col in sits_2.columns]

In [9]:
sits = agl.get.multiple_sits(gdf.iloc[0:2], satellite)

In [ ]:
def flatten_by_indexnum(df: pd.DataFrame, prefix: str = "") -> pd.DataFrame:
    if "indexnum" not in df.columns:
        raise ValueError("A coluna 'indexnum' não existe no DataFrame.")

    df = df.copy()
    df["seq"] = df.groupby("indexnum").cumcount()
    df_wide = df.pivot(index="indexnum", columns="seq").drop(columns="indexnum")
    df_wide.columns = [f"{prefix}{col}_{i}" for col, i in df_wide.columns]
    return df_wide.reset_index()

In [ ]:
sits

In [ ]:
flatten_by_indexnum(sits)

In [ ]:
flatten_by_indexnum(sits, "a")

In [ ]:
row = gdf.iloc[0]

imgs = agl.get.images(row.geometry, row.start_date, row.end_date, satellite)

In [ ]:
np.savez_compressed("tests/data/imgs/0_s2sr.npz", data=imgs)

In [ ]:
np.load("tests/data/imgs/0_s2sr.npz")

In [ ]:
np.array_equal(imgs, imgs)

In [ ]:
imgs_copy = imgs.copy()

In [ ]:
plt.imshow((imgs[3]/10000) * np.pi)

In [ ]:
sits = agl.get.sits(row.geometry, row.start_date, row.end_date, satellite)

In [ ]:
sits.columns = [column.split("_", 1)[1] for column in sits.columns.tolist()]

In [ ]:
def remove_underscore_in_df(df):
    df.columns = [column.split("_", 1)[1] for column in df.columns.tolist()]

In [ ]:
sits.head(5)